In [29]:
def Sigmoid(Z):
    return 1/(1+np.exp(-Z))

def Relu(Z):
    return np.maximum(0,Z)

def dRelu2(dZ, Z):    
    dZ[Z <= 0] = 0    
    return dZ

def dRelu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def dSigmoid(Z):
    s = 1/(1+np.exp(-Z))
    dZ = s * (1-s)
    return dZ

class dlnet:
    def __init__(self, x, y):
        self.debug = 0;
        self.X=x
        self.Y=y
        self.Yh=np.zeros((1,self.Y.shape[0])) 
        self.L=2
        self.dims = [3,2,1] 
        self.param = {}
        self.ch = {}
        self.grad = {}
        self.loss = []
        self.lr=0.003
        self.sam = self.Y.shape[0]
        self.threshold=0.5
        
    def nInit(self):    
        np.random.seed(1)
        self.param['W1'] = np.full((self.dims[1], self.dims[0]),0.5) 
        self.param['b1'] = np.zeros((self.dims[1], 1))        
        self.param['W2'] = np.full((self.dims[2], self.dims[1]),0.7) 
        self.param['b2'] = np.zeros((self.dims[2], 1))                
        return 

    def forward(self):    
        Z1 = self.param['W1'].dot(self.X) + self.param['b1'] 
        print('Z1',Z1)
        print('W1',self.param['W1'])
        
        print("b1:",self.param["b1"])
                
        A1 = Relu(Z1)
        print('A1',A1)
        self.ch['Z1'],self.ch['A1']=Z1,A1
        
        Z2 = self.param['W2'].dot(A1) + self.param['b2'] 
        print('Z2',Z2)
        print("W2:",self.param["W2"])
        print("b2:",self.param["b2"])
        A2 = Sigmoid(Z2)
        print('A2',A1)
        self.ch['Z2'],self.ch['A2']=Z2,A2

        self.Yh=A2
        loss=self.nloss(A2)
        print(loss)
        return self.Yh, loss

    def nloss(self,Yh):
        loss = (1./self.sam) * (-np.dot(self.Y,np.log(Yh).T) - np.dot(1-self.Y, np.log(1-Yh).T))
        print('Loss:',loss)
        return loss

    def backward(self):
        dLoss_Yh = - (np.divide(self.Y, self.Yh ) - np.divide(1 - self.Y, 1 - self.Yh))    
        print('dLoss_Yh',dLoss_Yh)
        dLoss_Z2 = dLoss_Yh * dSigmoid(self.ch['Z2'])
        print('dLoss_Z2',dLoss_Z2)
        dLoss_A1 = np.dot(self.param["W2"].T,dLoss_Z2)
        print('dLoss_A1',dLoss_A1)
        dLoss_W2 = 1./self.ch['A1'].shape[1] * np.dot(dLoss_Z2,self.ch['A1'].T)
        print('dLoss_W2',dLoss_W2)
        dLoss_b2 = 1./self.ch['A1'].shape[1] * np.dot(dLoss_Z2, np.ones([dLoss_Z2.shape[1],1]))
        print('dLoss_b2',dLoss_b2)
                            
        dLoss_Z1 = dLoss_A1 * dRelu(self.ch['Z1']) 
        print('dLoss_Z1')
        dLoss_A0 = np.dot(self.param["W1"].T,dLoss_Z1)
        print('dLoss_A0',dLoss_A0)
        dLoss_W1 = 1./self.X.shape[1] * np.dot(dLoss_Z1,self.X.T)
        print('dLoss_W1',dLoss_W1)
        dLoss_b1 = 1./self.X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1])) 
        print('dLoss_b1',dLoss_b1)
        
        self.param["W1"] = self.param["W1"] - self.lr * dLoss_W1
        self.param["b1"] = self.param["b1"] - self.lr * dLoss_b1
        self.param["W2"] = self.param["W2"] - self.lr * dLoss_W2
        self.param["b2"] = self.param["b2"] - self.lr * dLoss_b2
        
        return


    def pred(self,x, y):  
        self.X=x
        self.Y=y
        comp = np.zeros((1,x.shape[1]))
        pred, loss= self.forward()    
    
        for i in range(0, pred.shape[1]):
            if pred[0,i] > self.threshold: comp[0,i] = 1
            else: comp[0,i] = 0
    
        print("Acc: " + str(np.sum((comp == y)/x.shape[1])))
        
        return comp
    
    def gd(self,X, Y, iter = 1):
        np.random.seed(1)                         
    
        self.nInit()
    
        for i in range(0, iter):
            Yh, loss=self.forward()
            self.backward()
        
            if i % 1 == 0:
                
                print("W1:",nn.param["W1"])
                print("W2:",nn.param["W2"])
                print("b1:",nn.param["b1"])
                print("b2:",nn.param["b2"])
                self.loss.append(loss)

    
        return

In [30]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools
dataset=pd.DataFrame({'F1':[1,3,2,1],'F2':[7,6,2,2],'F3':[8,7,4,3],'Class':['A','A','B','B']})

dataset.iloc[:,3].replace('A',0,inplace=True)

dataset.iloc[:,3].replace('B',1,inplace=True)
names=dataset.columns[0:3]

scaler= MinMaxScaler()
scaled_dataset = scaler.fit_transform(dataset.iloc[:,0:3])
scaled_dataset = pd.DataFrame(scaled_dataset, columns=names)
scaled_dataset

x=scaled_dataset.iloc[:,0:3].values.transpose()
y=dataset.iloc[:,3].values.transpose()

print(x)
print(y)

[[0.  1.  0.5 0. ]
 [1.  0.8 0.  0. ]
 [1.  0.8 0.2 0. ]]
[0 0 1 1]


In [9]:
print(x.shape, y.shape)

(3, 4) (4,)


In [31]:
nn = dlnet(x,y)
nn.lr=0.07
nn.dims = [3, 2, 1]

In [32]:
nn.gd(x, y, iter = 1)

Z1 [[1.   1.3  0.35 0.  ]
 [1.   1.3  0.35 0.  ]]
W1 [[0.5 0.5 0.5]
 [0.5 0.5 0.5]]
b1: [[0.]
 [0.]]
A1 [[1.   1.3  0.35 0.  ]
 [1.   1.3  0.35 0.  ]]
Z2 [[1.4  1.82 0.49 0.  ]]
W2: [[0.7 0.7]]
b2: [[0.]]
A2 [[1.   1.3  0.35 0.  ]
 [1.   1.3  0.35 0.  ]]
Loss: [1.19039839]
[1.19039839]
dLoss_Yh [[ 5.05519997  7.17185845 -1.61262639 -2.        ]]
dLoss_Z2 [[ 0.80218389  0.86056613 -0.37989357 -0.5       ]]
dLoss_A1 [[ 0.56152872  0.60239629 -0.2659255  -0.35      ]
 [ 0.56152872  0.60239629 -0.2659255  -0.35      ]]
dLoss_W2 [[0.44698928 0.44698928]]
dLoss_b2 [[0.19571411]]
dLoss_Z1
dLoss_A0 [[ 0.56152872  0.60239629 -0.2659255   0.        ]
 [ 0.56152872  0.60239629 -0.2659255   0.        ]
 [ 0.56152872  0.60239629 -0.2659255   0.        ]]
dLoss_W1 [[0.11735839 0.26086144 0.24756516]
 [0.11735839 0.26086144 0.24756516]]
dLoss_b1 [[0.22449988]
 [0.22449988]]
W1: [[0.49178491 0.4817397  0.48267044]
 [0.49178491 0.4817397  0.48267044]]
W2: [[0.66871075 0.66871075]]
b1: [[-0.01571499]
 [

In [10]:
L=len([3,4,5,6])
for l in reversed(range(L-1)):
    print(l)

2
1
0


In [7]:
for i in range(1,L):
    print(i)

1
2
3
